In [1]:
import cv2
import os 
import numpy as np
import tensorflow as tf

In [2]:
def inference(input_tensor,train,regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights = tf.get_variable("weight",[5,5,3,32],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias",[32],initializer=tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
        
    with tf.name_scope('layer2-pool1'):
        pool1 = tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    
    with tf.variable_scope('layer3-conv2'):
        conv2_weights = tf.get_variable("weight",[5,5,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias",[64],initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
        
    with tf.name_scope('layer4-pool2'):
        pool2 = tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    
    with tf.variable_scope('layer5-conv3'):
        conv3_weights = tf.get_variable("weight",[3,3,64,128],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv3_biases = tf.get_variable("bias",[128],initializer=tf.constant_initializer(0.0))
        conv3 = tf.nn.conv2d(pool2,conv3_weights,strides=[1,1,1,1],padding='SAME')
        relu3 = tf.nn.relu(tf.nn.bias_add(conv3,conv3_biases))
        
    with tf.name_scope('layer6-pool3'):
        pool3 = tf.nn.max_pool(relu3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('layer7-conv4'):
        conv4_weights = tf.get_variable("weight",[3,3,128,256],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv4_biases = tf.get_variable("bias",[256],initializer=tf.constant_initializer(0.0))
        conv4 = tf.nn.conv2d(pool3,conv4_weights,strides=[1,1,1,1],padding='SAME')
        relu4 = tf.nn.relu(tf.nn.bias_add(conv4,conv4_biases))
        
    with tf.name_scope('layer8-pool4'):
        pool4 = tf.nn.max_pool(relu4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        pool_shape = pool4.get_shape().as_list()
        nodes = pool_shape[1]*pool_shape[2]*pool_shape[3]
        reshaped = tf.reshape(pool4,[-1,nodes])
        
    with tf.variable_scope('layer9-fc1'):
        fc1_weights = tf.get_variable("weight",[nodes,1024],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer is not None: 
            tf.add_to_collection('loss',regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias",[1024],initializer=tf.constant_initializer(0.0))
        fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_weights)+fc1_biases)
        if train:
            fc1 = tf.nn.dropout(fc1,0.5)
            
    with tf.variable_scope('layer10-fc2'):
        fc2_weights = tf.get_variable("weight",[1024,512],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer is not None: 
            tf.add_to_collection('loss',regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias",[512],initializer=tf.constant_initializer(0.0))
        fc2 = tf.nn.relu(tf.matmul(fc1,fc2_weights)+fc2_biases)
        if train:
            fc2 = tf.nn.dropout(fc2,0.5)
            
    with tf.variable_scope('layer11-fc3'):
        fc3_weights = tf.get_variable("weight",[512,2],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer is not None: 
            tf.add_to_collection('loss',regularizer(fc3_weights))
        fc3_biases = tf.get_variable("bias",[2],initializer=tf.constant_initializer(0.0))
        logit = tf.matmul(fc2,fc3_weights)+fc3_biases
        
    return logit

In [3]:
def get_data(path,has_glass):
    files = os.listdir(path)
    imgs = []
    labels = []
    for f in files:
        img = cv2.imread(path+f)
        img = cv2.resize(img,(100,100))
        imgs.append(img)
        if has_glass:
            labels.append(1)
        else:
            labels.append(0)
    return imgs,labels

In [4]:
has_path = r"E:/celeA/has/"
no_path = r"E:/celeA/no/"
img,label = get_data(has_path,True)
no_img,no_label = get_data(no_path,False)

In [5]:
img.extend(no_img)
label.extend(no_label)
#Y = np.eye(2)[label]
X = np.asarray(img,np.float32)
Y = np.asarray(label,np.int32)

In [6]:
len(X),len(Y)

(10000, 10000)

In [7]:
# 打乱顺序 
num_example=X.shape[0]
arr=np.arange(num_example)
np.random.shuffle(arr)
X=X[arr]
Y=Y[arr]

#将所有数据分为训练集和验证集
ratio=0.8
s=np.int(num_example*ratio)
x_train=X[:s]
y_train=Y[:s]
x_val=X[s:]
y_val=Y[s:]

In [8]:
#定义一个函数，按批次取数据
def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]

In [9]:
w = 100
h = 100
c = 3

n_epoch = 50                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
batch_size = 64
model_path = r'E:/celeA/model'
summary_path = r'E:/celeA/log'

with tf.name_scope('input'):
    #占位符
    x=tf.placeholder(tf.float32,shape=[None,w,h,c],name='x')
    y_=tf.placeholder(tf.int32,shape=[None,],name='y_')

regularizer = tf.contrib.layers.l2_regularizer(0.0001)
logits = inference(x,True,regularizer)
global_step = tf.Variable(0,trainable=False)
#(小处理)将logits乘以1赋值给logits_eval，定义name，方便在后续调用模型时通过tensor名字调用输出tensor
b = tf.constant(value=1,dtype=tf.float32)
logits_eval = tf.multiply(logits,b,name='logits_eval') 

with tf.name_scope('train'):
    with tf.name_scope('corss_entropy'):
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_))
        tf.summary.scalar('corss_entropy',loss)
    with tf.name_scope('train_op'):
        train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss,global_step=global_step)
    
with tf.name_scope('acc'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.cast(tf.argmax(logits,1),tf.int32), y_)    
    with tf.name_scope('accuary'):
        acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuary',acc)
        
merged = tf.summary.merge_all()
saver = tf.train.Saver()
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(summary_path+'/train',tf.get_default_graph())
    val_writer = tf.summary.FileWriter(summary_path+'/val')
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epoch):
        print("====epoch %d====="%epoch)
        #training
        train_loss,train_acc,n_batch = 0,0,0
        for x_train_a,y_train_a in minibatches(x_train, y_train, batch_size, shuffle=True):
            train_sum,_,err,ac=sess.run([merged,train_op,loss,acc], feed_dict={x: x_train_a, y_: y_train_a})
            train_writer.add_summary(train_sum, epoch)
            train_loss += err; train_acc += ac; n_batch += 1
        print("   train loss: %f" % (np.sum(train_loss)/ n_batch))
        print("   train acc: %f" % (np.sum(train_acc)/ n_batch))

        #validation
        val_loss, val_acc, n_batch = 0, 0, 0
        for x_val_a, y_val_a in minibatches(x_val, y_val, batch_size, shuffle=False):
            val_sum, err, ac = sess.run([merged,loss,acc], feed_dict={x: x_val_a, y_: y_val_a})
            val_writer.add_summary(val_sum, epoch)
            val_loss += err; val_acc += ac; n_batch += 1
        print("   validation loss: %f" % (np.sum(val_loss)/ n_batch))
        print("   validation acc: %f" % (np.sum(val_acc)/ n_batch))
        
        saver.save(sess,model_path,global_step=global_step)
    train_writer.close()
    val_writer.close()

====epoch 0=====
   train loss: 708.027806
   train acc: 0.654500
   validation loss: 24.983445
   validation acc: 0.733871
====epoch 1=====
   train loss: 17.637438
   train acc: 0.742000
   validation loss: 14.333073
   validation acc: 0.764113
====epoch 2=====
   train loss: 9.102842
   train acc: 0.784625
   validation loss: 7.190975
   validation acc: 0.792843
====epoch 3=====
   train loss: 5.600294
   train acc: 0.817750
   validation loss: 5.324786
   validation acc: 0.809476
====epoch 4=====
   train loss: 3.956611
   train acc: 0.830625
   validation loss: 3.901650
   validation acc: 0.810484
====epoch 5=====
   train loss: 2.786851
   train acc: 0.834625
   validation loss: 2.432648
   validation acc: 0.849798
====epoch 6=====
   train loss: 1.912772
   train acc: 0.857875
   validation loss: 2.039976
   validation acc: 0.834173
====epoch 7=====
   train loss: 1.495205
   train acc: 0.861875
   validation loss: 1.612010
   validation acc: 0.849294
====epoch 8=====
   train l